# 第14章：システム化・回帰・クラスタリング

## 14.1 ラボラトリー社の商品をひとつ取り上げて、売上予測の重回帰モデルを作ってみよう。

### 【解答例】

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df1 = pd.read_csv("../input/gi_train_mm10.csv") # 10月のデータ
df2 = pd.read_csv("../input/gi_train_mm11.csv") # 11月のデータ

df_tmp = pd.concat([df1, df2])
df = df_tmp[df_tmp["customer_id"].notna()].copy()

- ラボラトリー社商品の購入数ランキング

In [ ]:
df[df["company"]=="ラボラトリー"].groupby("product_name",as_index=False)["buy_flag"].sum().sort_values("buy_flag",ascending=False).head(10)

- ここでは、購入数の最も多い「雪のしずく550ml」で重回帰モデルを作成するものとする。
- ただ、現実的にはデータポイントが7点（2020/10/26～2020/11/01）しかない為、以下の制約を設けるものとする。
    - リークは考えない
        - 未来の売上予測する際にも、立寄人数や接触人数等の情報がわかるものとする
    - 時系列モデルは考えない
    - 多重共線性も考えない

In [ ]:
# 「雪のしずく550ml」の日毎の購入数を集計する
day_buy_count_df = df[df["product_name"]=="雪のしずく550ml"].groupby("event_day",as_index=False)["buy_flag"].sum()
day_buy_count_df.set_index("event_day",inplace=True)

#----------------------------------------------------------------------------------------------------------------------- 
# 特徴量
## 立寄人数
tachiyori_df = df[(df["event_type"]==1)].groupby("event_day",as_index=False)["customer_id"].nunique()
tachiyori_df.rename(columns={"customer_id":"立寄人数"},inplace=True)
tachiyori_df.set_index("event_day",inplace=True)

## 滞在者人数
taizai_df = df[(df["event_type"]==1)&(df["time_duration"]>=10)].groupby("event_day",as_index=False)["customer_id"].nunique()
taizai_df.rename(columns={"customer_id":"滞在人数"},inplace=True)
taizai_df.set_index("event_day",inplace=True)

## 接触者人数
touch_df = df[df["num_touch"]==1].groupby("event_day",as_index=False)["customer_id"].nunique()
touch_df.rename(columns={"customer_id":"接触人数"},inplace=True)
touch_df.set_index("event_day",inplace=True)

## 購入者人数
buy_df = df[df["buy_flag"]==1].groupby("event_day",as_index=False)["customer_id"].nunique()
buy_df.rename(columns={"customer_id":"購入人数"},inplace=True)
buy_df.set_index("event_day",inplace=True)

## 日毎の平均滞在時間
timeduration_df = df[(df["event_type"]==1)].groupby("event_day",as_index=False)["time_duration"].mean()
timeduration_df.set_index("event_day",inplace=True)

## 男女比率
gender_rate_df = df[(df["event_type"]==1)].pivot_table(index="event_day",columns="gender",values="customer_id",aggfunc="count")
gender_rate_df["男性割合"] = gender_rate_df["man"] / gender_rate_df.sum(axis=1)

#----------------------------------------------------------------------------------------------------------------------- 

# 上記のデータフレームを結合する
all_df = pd.concat([day_buy_count_df,tachiyori_df,taizai_df,touch_df,buy_df,timeduration_df,gender_rate_df["男性割合"]],axis=1)
all_df

In [ ]:
all_df.corr()

- statsmodelsを使用して重回帰分析を行う

In [ ]:
import statsmodels.api as sm

df_y = all_df[["buy_flag"]]
df_X = all_df[["time_duration","男性割合"]]

df_X = sm.add_constant(df_X)

model = sm.OLS(df_y, df_X)
result = model.fit()
print(result.summary())

### 【解説】

- ここで使用したStatsModelsは、統計モデルを用いて推定や検定、探索が出来るPythonライブラリとなっています。書籍本文では機械学習用のライブラリとしてscikit-learnを使用したが、それと似たライブラリとなっています。
- しかし、ある程度お察しかと思いますが、このモデルはあまり意味がありません。冒頭に制約としても記載しましたが、リークは考えない、時系列モデルは考えない、多重共線性も考えない、さらにデータポイントも7点しかないことから、ほとんど実用性が無いモデルとなっています。実際にモデル構築結果の決定係数（R-squared）も0.315となっており精度も低いです。
- この章末問題を通して感じてもらいたかったのは、回帰のモデルを構築することは可能ではあるが、現在手元にあるデータだけではあまり精度のよいモデルが出来ない（意味のないモデルが出来てしまう）ことがある、ということを実感して頂くことにあります。逆に言えば、もっとデータを量と質を高めれば、意味があり精度の高い回帰モデルを構築することが出来そうだ、ということがお分かりいただけたのではないでしょうか。

## 14.2 k-means法を使ってクラスタリングを行ってみよう。

### 【解答例】

- クラスタリングを行うにあたって「何をクラスタリング（分類）するか」を考えてみよう。
- 大きく分けて以下の2つの種類がある。
    - 人物（ショッパー）のクラスタリング
    - 商品のクラスタリング
- この2つについてそれぞれ考えてみよう。

#### 人物（ショッパー）のクラスタリング

- 事前知識として、プログラムでは以下の処理を行っている。
    - 名義変数をダミー変数化する
        - k-meansが文字列の変数を扱えないため
    - データを標準化する
        - 標準偏差が大きい変数ほどクラスタの分類に大きく影響し、標準化の有無によって分類結果が異なる可能性があるため
    - エルボー法
        - 最適なクラスター数を求めるための手法
        - クラスターごとのSSE値(クラスタ内誤平方和)をプロットした図

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# ショッパー毎のフレーム滞在時間を集計
tmp1 = df[df["event_type"]==1][["customer_id","gender","time_duration"]]
tmp1.set_index("customer_id",inplace=True)

# ショッパー毎の接触・購入回数を集計
tmp2 = df.groupby("customer_id").agg({"num_touch":"sum","buy_flag":"sum"})

# 上記のデータフレームを結合
customer_df = pd.concat([tmp1,tmp2],axis=1)

# 名義変数をダミー変数化
customer_df = pd.get_dummies(customer_df)

# データの標準化
sc = StandardScaler()
customer_df_sc = sc.fit_transform(customer_df)
customer_df_sc = pd.DataFrame(customer_df_sc, columns=customer_df.columns)
customer_df_sc

In [ ]:
# エルボー法を実行
sum_of_squared_errors = []
for i in range(1, 11):
    model = KMeans(n_clusters=i, random_state=0, init="random")
    model.fit(customer_df_sc)
    sum_of_squared_errors.append(model.inertia_)  # 損失関数の値を保存

plt.plot(range(1, 11), sum_of_squared_errors, marker="o")
plt.xlabel("number of clusters")
plt.ylabel("sum of squared errors")
plt.show()

In [ ]:
# 上記のエルボー法を参考に、クラスター数を「3」でk-meansクラスタリングを実行
model = KMeans(n_clusters=3, random_state=1)
model.fit(customer_df_sc)
cluster = model.labels_

# データバーを用いて、各クラス毎の平均値を可視化
customer_df["クラス"] = cluster
customer_df.groupby("クラス").mean().style.bar(axis=0,width=90)

- k-meansを用いた結果、人物（ショッパー）を以下の3つのグループに分けられそうなことが分かった。
  - クラス0：100％男性のグループで滞在時間が比較的短く(31秒程度)、接触・購入回数も少ない
  - クラス1：100％女性のグループで滞在時間が比較的短く(35秒程度)、接触・購入回数も少ない
  - クラス2：男女混合のグループで滞在時間が比較的長く(77秒)、接触・購入回数が多い

#### 商品のクラスタリング

- 方法は人物（ショッパー）のクラスタリングの時とほぼ同様で、ダミー変数化と標準化を行い、エルボー法を用いて最適なクラスター数を検討する。

In [ ]:
# 商品毎の接触・購入回数を集計
tmp1 = df.groupby("product_name").agg({"num_touch":"sum","buy_flag":"sum"})

# 商品毎の購入・男女比率を計算
tmp2 = df[df["buy_flag"]==1].pivot_table(index="product_name",columns="gender",values="buy_flag",aggfunc="count")
tmp2["購入男性割合"] = tmp2["man"] / (tmp2["man"]+tmp2["woman"])
tmp2["購入男性割合"] = tmp2["購入男性割合"].fillna(0)

# 上記のデータフレームを結合
product_df = pd.concat([tmp1,tmp2["購入男性割合"]],axis=1)

# データの標準化
sc = StandardScaler()
product_df_sc = sc.fit_transform(product_df)
product_df_sc = pd.DataFrame(product_df_sc, columns=product_df.columns)
product_df_sc

In [ ]:
# エルボー法を実行
sum_of_squared_errors = []
for i in range(1, 11):
    model = KMeans(n_clusters=i, random_state=0, init="random")
    model.fit(product_df_sc)
    sum_of_squared_errors.append(model.inertia_)  # 損失関数の値を保存

plt.plot(range(1, 11), sum_of_squared_errors, marker="o")
plt.xlabel("number of clusters")
plt.ylabel("sum of squared errors")
plt.show()

In [ ]:
# 上記のエルボー法を参考に、クラスター数を「3」でk-meansクラスタリングを実行
model = KMeans(n_clusters=3, random_state=1)
model.fit(product_df_sc)
cluster = model.labels_

# データバーを用いて、各クラス毎の平均値を可視化
product_df["クラス"] = cluster
product_df.groupby("クラス").mean().style.bar(axis=0,width=90)

- k-meansを用いた結果、商品を以下の3つのグループに分けられそうなことが分かった
  - クラス0：男性の購入割合が低く(30％程度)、接触・購入回数が少ない商品群
  - クラス1：購入の男女比率はほぼ等しく、接触・購入回数が多い商品群
  - クラス2：男性の購入割合がやや高く(61％程度)、接触・購入回数が少ない商品群

- 参考の為に、それぞれのクラスタに所属する商品の商品名を一覧表示してみる

In [ ]:
print("クラス0に所属する商品一覧")
print(product_df[product_df["クラス"]==0].index.tolist())
print("-"*80)
#----------------------------------------------------------------------------------------------------------------------- 
print("クラス1に所属する商品一覧")
print(product_df[product_df["クラス"]==1].index.tolist())
print("-"*80)
#----------------------------------------------------------------------------------------------------------------------- 
print("クラス2に所属する商品一覧")
print(product_df[product_df["クラス"]==2].index.tolist())
print("-"*80)

## 14.3 回帰やクラスタリングがどのようなシチュエーションで使えそうか考えてみよう。

### 【解答例】

#### 回帰

- 回帰は需要予測で使えそうだ。
- 例えば飲料だと、事前に「どれくらい売れるか」を予測できると、供給過多による無駄な廃棄が少なくなったり、供給不足による欠品が少なくなったりして、経営的にも良い効果がありそうだ。

#### クラスタリング

- クラスタリングは商品の分類に使えそうだ。
- 前提として、商品はサブカテゴリ分類というものがある。飲料であれば、水、コーヒー、炭酸飲料、果汁飲料、などである。ただ、それはどちらかというと商品の成分や味などに起因した分類であり、消費者ニーズとは必ずしも合致していない。例えば「運動した後に飲みたいもの」としては水の場合もあれば炭酸飲料の場合もあるだろう。
- そして、消費者ニーズに照らし合わせた分類（クラスタリング）が出来れば、商品ラインナップや販促活動の改善にも繋げることが出来そうだ。例えば、「運動した後に飲みたいものはこの商品群A」ということが分かれば、公園や運動場などに設置された自販機やそこに近い店舗などでは、商品群Aを多く配荷するといった対応を取ることが可能になりそうだ。

### 【解説】

- 書籍本編でも扱った二値分類にしろ、本章で取り上げた回帰やクラスタリングにしろ、機械学習はあくまでも手段です。それが普段行っている事業にどのように活用できるかを考えることが大切です。
- 今回はシチュエーションとしてそれぞれ1つずつ（需要予測、商品の分類）挙げてみましたが、もちろん使えるシチュエーションは様々ですし、企業や業態によって様々です。状況による、ということも多いでしょう。そこで大切なのがビジネス理解です。ビジネスの現状からブレークダウンしていくことで「課題は何か」を突き止め、そこに回帰やクラスタリングといった手法がどのように適用できそうかを結び付けていくと良いでしょう。